In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

2024-10-17 18:06:38.821577: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-17 18:06:39.740735: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Num GPUs Available:  0


2024-10-17 18:06:43.364150: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: UNKNOWN ERROR (34)


# 1. Procesamiento de datos

In [ ]:
pip install scikit-learn

In [1]:
import pandas as pd
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import LabelEncoder

#modelos
import tensorflow as tf
from tensorflow.keras import layers, models

2024-10-17 18:56:57.541736: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-17 18:56:57.587469: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
#cargar dataset con los datos etiquetados
annotations = pd.read_csv('workspace/annotations/_annotations.csv')

In [3]:
#crear un codificador de clases
le = LabelEncoder()
annotations['class'] = le.fit_transform(annotations['class'])

In [4]:
# cargar imagenes, cajas etiquetadas y etiqueta de la caja
def load_image_and_boxes(annotations, image_dir):
    images = []
    boxes = []
    labels = []
    
    for index, row in annotations.iterrows():
        image_path = os.path.join(image_dir, row['filename'])
        image = cv2.imread(image_path)
        height, width, _ = image.shape
        
        # Normalizar bounding boxes
        xmin = row['xmin'] / width
        ymin = row['ymin'] / height
        xmax = row['xmax'] / width
        ymax = row['ymax'] / height
        
        images.append(image)
        boxes.append([xmin, ymin, xmax, ymax])
        labels.append(row['class'])
    
    return np.array(images), np.array(boxes), np.array(labels)


In [5]:
image_dir = 'workspace/images/train/'
images, boxes, labels = load_image_and_boxes(annotations, image_dir)

# 2. Crear el Modelo CNN

In [6]:
#construir modelo cnn
def build_cnn_model(input_shape, num_classes):
    input_image = layers.Input(shape=input_shape)
    
    # Extracción de características (Convoluciones)
    x = layers.Conv2D(32, (3, 3), activation='relu')(input_image)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    
    x = layers.Conv2D(64, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    
    x = layers.Conv2D(128, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    
    x = layers.Flatten()(x)
    
    #predicción del Bounding Box (4 valores: xmin, ymin, xmax, ymax)
    box_output = layers.Dense(4, activation='sigmoid', name='box_output')(x)
    
    #predicción de la clase del objeto
    class_output = layers.Dense(num_classes, activation='softmax', name='class_output')(x)
    
    model = models.Model(inputs=input_image, outputs=[box_output, class_output])
    return model

In [7]:
# definir modelo
input_shape = (1080, 1080, 3)  # Tamaño de las imágenes (ajusta según sea necesario)
num_classes = len(le.classes_)
model = build_cnn_model(input_shape, num_classes)

# Compilar el modelo
model.compile(optimizer='adam', 
              loss={'box_output': 'mean_squared_error', 'class_output': 'sparse_categorical_crossentropy'}, 
              metrics={'class_output': 'accuracy'})

model.summary()

2024-10-17 18:57:23.591602: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: UNKNOWN ERROR (34)


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 1080,      │          0 │ -                 │
│ (InputLayer)        │ 1080, 3)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 1078,      │        896 │ input_layer[0][0] │
│                     │ 1078, 32)         │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 539, 539,  │          0 │ conv2d[0][0]      │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 537, 537,  │     18,496 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 268, 268,  │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 266, 266,  │     73,856 │ max_pooling2d_1[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 133, 133,  │          0 │ conv2d_2[0][0]    │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 2264192)   │          0 │ max_pooling2d_2[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ box_output (Dense)  │ (None, 4)         │  9,056,772 │ flatten[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ class_output        │ (None, 2)         │  4,528,386 │ flatten[0][0]     │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 13,678,406 (52.18 MB)

 Trainable params: 13,678,406 (52.18 MB)

 Non-trainable params: 0 (0.00 B)

# 3. Preprocesamiento y redimensionamiento de Imágenes

In [8]:
#preprocesar imagenes para asignar a la red
def preprocess_images(images, target_size):
    resized_images = []
    for image in images:
        resized_image = cv2.resize(image, target_size)
        resized_image = resized_image / 255.0  #normalizar los valores de los píxeles
        resized_images.append(resized_image)
    return np.array(resized_images)

target_size = (1080, 1080)  #ajustar al tamaño de la entrada de la red
images_resized = preprocess_images(images, target_size)

# 4. Entrenamiento del modelo CNN

In [9]:
history = model.fit(images_resized, {'box_output': boxes, 'class_output': labels}, 
                    epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
43/43 ━━━━━━━━━━━━━━━━━━━━ 105s 2s/step - class_output_accuracy: 0.6482 - loss: 5.9786 - val_class_output_accuracy: 0.9150 - val_loss: 0.4198
Epoch 2/10
43/43 ━━━━━━━━━━━━━━━━━━━━ 100s 2s/step - class_output_accuracy: 0.8004 - loss: 0.5348 - val_class_output_accuracy: 0.9150 - val_loss: 0.3743
Epoch 3/10
43/43 ━━━━━━━━━━━━━━━━━━━━ 100s 2s/step - class_output_accuracy: 0.8105 - loss: 0.5108 - val_class_output_accuracy: 0.9150 - val_loss: 0.4232
Epoch 4/10
43/43 ━━━━━━━━━━━━━━━━━━━━ 99s 2s/step - class_output_accuracy: 0.8006 - loss: 0.5369 - val_class_output_accuracy: 0.9150 - val_loss: 0.4106
Epoch 5/10
43/43 ━━━━━━━━━━━━━━━━━━━━ 99s 2s/step - class_output_accuracy: 0.7960 - loss: 0.5329 - val_class_output_accuracy: 0.9150 - val_loss: 0.3675
Epoch 6/10
43/43 ━━━━━━━━━━━━━━━━━━━━ 99s 2s/step - class_output_accuracy: 0.7881 - loss: 0.5345 - val_class_output_accuracy: 0.9150 - val_loss: 0.3596
Epoch 7/10
43/43 ━━━━━━━━━━━━━━━━━━━━ 99s 2s/step - class_output_accuracy: 0.8028 - l

# 5. Evaluación de Modelo

In [ ]:
# Predecir en una nueva imagen
new_image = cv2.imread('path_to_new_image.jpg')
new_image_resized = preprocess_images([new_image], target_size)

predictions = model.predict(new_image_resized)
predicted_box = predictions[0][0]  # Coordenadas del bounding box
predicted_class = np.argmax(predictions[1][0])  # Clase predicha

print(f"Bounding Box: {predicted_box}")
print(f"Clase Predicha: {le.inverse_transform([predicted_class])[0]}")


In [ ]:
import matplotlib.pyplot as plt

# Visualizar el resultado
plt.imshow(cv2.cvtColor(new_image, cv2.COLOR_BGR2RGB))
xmin, ymin, xmax, ymax = predicted_box
height, width, _ = new_image.shape
plt.gca().add_patch(plt.Rectangle((xmin * width, ymin * height), (xmax - xmin) * width, (ymax - ymin) * height, 
                                  edgecolor='r', facecolor='none', lw=2))
plt.show()
